In [3]:
import pandas as pd
import re

# --- 准备工作：定义数据清洗和映射所需的函数与字典 ---

def standardize_province(name):
    """标准化中文省份名称，去除后缀以便合并。"""
    if isinstance(name, str):
        return re.sub(r'省|市|自治区|维吾尔|壮族|回族', '', name).strip()
    return name

# 中文省份名 -> 两位行政区代码 的映射字典
province_codes_map = {
    '北京': '11', '天津': '12', '河北': '13', '山西': '14', '内蒙古': '15', '辽宁': '21',
    '吉林': '22', '黑龙江': '23', '上海': '31', '江苏': '32', '浙江': '33', '安徽': '34',
    '福建': '35', '江西': '36', '山东': '37', '河南': '41', '湖北': '42', '湖南': '43',
    '广东': '44', '广西': '45', '海南': '46', '重庆': '50', '四川': '51', '贵州': '52',
    '云南': '53', '西藏': '54', '陕西': '61', '甘肃': '62', '青海': '63', '宁夏': '64', '新疆': '65'
}

print("--- 开始集成第二产业增加值数据 ---")

# --- 步骤 1: 加载总表和新数据 ---
try:
    df_main = pd.read_csv('/Users/yumanlou/Library/CloudStorage/OneDrive-email.cufe.edu.cn/2025/第五学期/论文/ESG/carbon_emission_finance/data/data1.2_ctrl.csv')
    # 第二产业数据文件有3行无效表头，需要跳过
    df_sec_ind_wide = pd.read_csv('/Users/yumanlou/Library/CloudStorage/OneDrive-email.cufe.edu.cn/2025/第五学期/论文/ESG/carbon_emission_finance/data/rawdata/二产分省年度数据_utf8.csv', header=3)
    print("总表 'data1.2_ctrl.csv' 和新数据 '分省年度数据.csv' 加载成功。")
except FileNotFoundError as e:
    print(f"错误: 找不到文件 {e}。请确保文件都在Notebook所在的文件夹中。")

# --- 步骤 2: 处理第二产业数据 (宽表转长表) ---
df_sec_ind_wide.rename(columns={'地区': 'province_chinese'}, inplace=True)
df_sec_ind_long = df_sec_ind_wide.melt(id_vars=['province_chinese'], var_name='year', value_name='secondary_industry_value')

# 清洗年份和省份列
df_sec_ind_long['year'] = df_sec_ind_long['year'].str.replace('年', '').astype(int)
df_sec_ind_long['province_chinese'] = df_sec_ind_long['province_chinese'].apply(standardize_province)
df_sec_ind_long.dropna(subset=['province_chinese', 'secondary_industry_value'], inplace=True)

# 根据中文名创建用于匹配的“两位省份代码”
df_sec_ind_long['province_code_2digit'] = df_sec_ind_long['province_chinese'].map(province_codes_map)
print("第二产业数据已处理为长表格式。")


# --- 步骤 3: 从总表的UID中解析出匹配键 ---
uid_str = df_main['uid'].astype(str).str.zfill(4)
df_main['province_code_2digit'] = uid_str.str[:2]
# 我们使用总表中已有的'year'列进行匹配，以确保一致性
print("已从总表 'uid' 中解析出省份代码。")


# --- 步骤 4: 基于解析出的键，将第二产业数据合并到总表 ---
# 使用左合并，以总表为基础
final_df = pd.merge(
    df_main,
    df_sec_ind_long[['province_code_2digit', 'year', 'secondary_industry_value']],
    on=['province_code_2digit', 'year'],
    how='left'
)
print("\n--- ✨ 第二产业数据集成完成！ ---")


# --- 步骤 5: 清理和整理 ---
# 删除为合并而创建的临时列
final_df.drop(columns=['province_code_2digit'], inplace=True)

# 调整新列的位置，使其更美观（可选）
if 'secondary_industry_value' in final_df.columns:
    sec_ind_col = final_df.pop('secondary_industry_value')
    # 把它放在 gdp 列的后面
    if 'gdp' in final_df.columns:
        final_df.insert(final_df.columns.get_loc('gdp') + 1, 'secondary_industry_value', sec_ind_col)
    else: # 如果没有gdp列，就放在year后面
        final_df.insert(final_df.columns.get_loc('year') + 1, 'secondary_industry_value', sec_ind_col)

print("最终数据表预览 (请检查 'secondary_industry_value' 列是否有数据):")
display(final_df.head())


# --- 步骤 6: 导出最终的CSV文件 ---
output_filename = 'final_data_with_secondary_industry.csv'
final_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"\n--- ✅ 导出成功！ ---")
print(f"包含第二产业增加值的最终面板数据已保存为文件: '{output_filename}'")

--- 开始集成第二产业增加值数据 ---
总表 'data1.2_ctrl.csv' 和新数据 '分省年度数据.csv' 加载成功。
第二产业数据已处理为长表格式。
已从总表 'uid' 中解析出省份代码。

--- ✨ 第二产业数据集成完成！ ---
最终数据表预览 (请检查 'secondary_industry_value' 列是否有数据):


,uid,province_id,province,year,co2_emissions,gdp,secondary_industry_value,green_invention_patents,green_utility_patents,credit,...,z_credit,z_bond,z_investment,z_insurance,z_equity,z_fund,z_carbon_finance,Factor_1,Factor_2,Factor_3
0,3100,1,Shanghai,2000,71538176,4880.0000,2215.8,98,109,0.024,...,-1.502512,-1.757816,-0.290447,-0.963996,-1.787280,-1.787502,0.296368,-1.821079,-0.159981,-0.341426
1,3101,1,Shanghai,2001,69645920,5348.1001,2413.8,165,202,0.024,...,-1.502512,-1.673861,-0.341557,-0.842728,-1.545422,-1.787502,0.302489,-1.744686,-0.131432,-0.358423
2,3102,1,Shanghai,2002,80866288,5908.1001,2635.3,272,230,0.025,...,-1.380046,-1.380017,-0.329291,-0.873045,-1.464803,-1.329610,0.187003,-1.475053,-0.215530,-0.363260
3,3103,1,Shanghai,2003,92319352,6952.5000,3239.6,382,254,0.024,...,-1.502512,-1.254084,-0.298624,-0.903362,-1.303564,-1.146453,0.073289,-1.376330,-0.248339,-0.386201
4,3104,1,Shanghai,2004,108254088,8309.4004,3872.2,451,354,0.025,...,-1.380046,-1.044195,-0.370179,-0.782093,-1.303564,-0.963296,-0.000802,-1.249054,-0.243342,-0.446714



--- ✅ 导出成功！ ---
包含第二产业增加值的最终面板数据已保存为文件: 'final_data_with_secondary_industry.csv'


In [6]:
import pandas as pd
import re

# --- 准备工作：定义数据清洗和映射所需的函数与字典 ---

def standardize_province(name):
    """标准化中文省份名称，去除后缀以便合并。"""
    if isinstance(name, str):
        return re.sub(r'省|市|自治区|维吾尔|壮族|回族', '', name).strip()
    return name

# 中文省份名 -> 两位行政区代码 的映射字典
province_codes_map = {
    '北京': '11', '天津': '12', '河北': '13', '山西': '14', '内蒙古': '15', '辽宁': '21',
    '吉林': '22', '黑龙江': '23', '上海': '31', '江苏': '32', '浙江': '33', '安徽': '34',
    '福建': '35', '江西': '36', '山东': '37', '河南': '41', '湖北': '42', '湖南': '43',
    '广东': '44', '广西': '45', '海南': '46', '重庆': '50', '四川': '51', '贵州': '52',
    '云南': '53', '西藏': '54', '陕西': '61', '甘肃': '62', '青海': '63', '宁夏': '64', '新疆': '65'
}

print("--- 开始根据 'energystruct.csv' 集成“煤炭占比”数据 ---")

# --- 步骤 1: 加载总表和更新后的新数据 ---
try:
    df_main = pd.read_csv('/Users/yumanlou/Library/CloudStorage/OneDrive-email.cufe.edu.cn/2025/第五学期/论文/ESG/carbon_emission_finance/data/final_data_with_secondary_industry.csv')
    df_coal_share = pd.read_csv('/Users/yumanlou/Library/CloudStorage/OneDrive-email.cufe.edu.cn/2025/第五学期/论文/ESG/carbon_emission_finance/data/energystruct.csv')
    print("总表和新的煤炭占比数据 'energystruct.csv' 加载成功。")
except FileNotFoundError as e:
    print(f"错误: 找不到文件 {e}。请确保文件都在Notebook所在的文件夹中。")

# --- 步骤 2: 准备“煤炭占比”数据 (根据新文件结构调整) ---

# 新文件加载后可能有一个多余的、未命名的列，我们检查并删除它
if 'Unnamed: 0' in df_coal_share.columns:
    df_coal_share.drop(columns=['Unnamed: 0'], inplace=True)
elif '\\' in df_coal_share.columns: # 有时可能会读作'\'
    df_coal_share.drop(columns=['\\'], inplace=True)

# 重命名列以便于操作
df_coal_share.rename(columns={
    '省份': 'province_chinese',
    '能源消费结构（煤炭占比）': 'coal_share_pctg'
}, inplace=True)

# 清洗中文省份名
df_coal_share['province_chinese'] = df_coal_share['province_chinese'].apply(standardize_province)

# 根据中文名创建用于匹配的“两位省份代码”
df_coal_share['province_code_2digit'] = df_coal_share['province_chinese'].map(province_codes_map)
print("煤炭占比数据已处理完毕。")


# --- 步骤 3: 从总表的UID中解析出匹配键 ---
uid_str = df_main['uid'].astype(str).str.zfill(4)
df_main['province_code_2digit'] = uid_str.str[:2]
print("已从总表 'uid' 中解析出省份代码。")


# --- 步骤 4: 基于解析出的键，将煤炭占比数据合并到总表 ---
# 使用左合并，以总表为基础，确保总表行数不变
final_df = pd.merge(
    df_main,
    df_coal_share[['province_code_2digit', 'year', 'coal_share_pctg']],
    on=['province_code_2digit', 'year'],
    how='left'
)
print("\n--- ✨ “煤炭占比”数据集成完成！ ---")


# --- 步骤 5: 清理和验证 ---
# 删除为合并而创建的临时列
final_df.drop(columns=['province_code_2digit'], inplace=True)

print("最终数据表预览 (请检查 'coal_share_pctg' 列是否有数据):")
# 由于煤炭占比数据是2003-2022年，我们特意查看2003年的数据以验证合并是否成功
display(final_df[final_df['year'] == 2003].head())


# --- 步骤 6: 导出最终的CSV文件 ---
output_filename = 'final_data.1.3.csv'
final_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"\n--- ✅ 导出成功！ ---")
print(f"包含所有控制变量的最终面板数据已保存为文件: '{output_filename}'")

--- 开始根据 'energystruct.csv' 集成“煤炭占比”数据 ---
总表和新的煤炭占比数据 'energystruct.csv' 加载成功。
煤炭占比数据已处理完毕。
已从总表 'uid' 中解析出省份代码。

--- ✨ “煤炭占比”数据集成完成！ ---
最终数据表预览 (请检查 'coal_share_pctg' 列是否有数据):


,uid,province_id,province,year,co2_emissions,gdp,sec_val,green_invention_patents,green_utility_patents,credit,...,z_bond,z_investment,z_insurance,z_equity,z_fund,z_carbon_finance,Factor_1,Factor_2,Factor_3,coal_share_pctg
3,3103,1,Shanghai,2003,92319352,6952.5000,3239.6,382,254,0.024,...,-1.254084,-0.298624,-0.903362,-1.303564,-1.146453,0.073289,-1.376330,-0.248339,-0.386201,0.299557
27,5303,2,Yunnan,2003,94204008,2673.6001,1059.8,66,47,0.027,...,-1.338039,-0.069648,-0.600190,-1.222944,-1.421188,-0.210835,-1.329980,-0.233613,-0.420211,0.639164
51,1503,3,Neimenggu,2003,193669040,2415.1001,840.9,15,36,0.026,...,-1.128151,0.040751,-0.206067,-1.303564,-1.054874,-0.150112,-1.225849,-0.140179,-0.330963,0.694738
75,1103,4,Beijing,2003,49524960,5400.1001,1456.4,715,378,0.025,...,-1.380017,-0.274091,-0.812411,-1.303564,-1.421188,0.355319,-1.465858,-0.118374,-0.234050,0.429326
99,2203,5,Jilin,2003,77726608,2169.7000,805.7,64,69,0.025,...,-1.086173,0.028485,-0.600190,-0.981086,-1.146453,0.306998,-1.162050,-0.226210,-0.070863,0.523261



--- ✅ 导出成功！ ---
包含所有控制变量的最终面板数据已保存为文件: 'final_data.1.3.csv'


In [7]:
import pandas as pd
import numpy as np

print("--- 开始集成“人口与城镇化”数据 ---")

# --- 步骤 1: 加载总表和新的人口数据 ---
try:
    df_main = pd.read_csv('/Users/yumanlou/Library/CloudStorage/OneDrive-email.cufe.edu.cn/2025/第五学期/论文/ESG/carbon_emission_finance/data/final_data.1.3.csv')
    df_pop = pd.read_csv('/Users/yumanlou/Library/CloudStorage/OneDrive-email.cufe.edu.cn/2025/第五学期/论文/ESG/carbon_emission_finance/data/pop.csv')
    print("总表和人口数据加载成功。")
except FileNotFoundError as e:
    print(f"错误: 找不到文件 {e}。请确保文件都在Notebook所在的文件夹中。")

# --- 步骤 2: 准备“人口数据” ---

# 重命名列以便于操作
df_pop.rename(columns={
    '省份代码': 'province_code_2digit',
    '年份': 'year',
    '年末常住人口/万人': 'population',
    '城镇人口/万人': 'urban_population'
}, inplace=True)

# 将 province_code_2digit 转换为与 UID 逻辑一致的两位字符串格式 (例如 11 -> "11")
df_pop['province_code_2digit'] = df_pop['province_code_2digit'].astype(str).str.zfill(2)

# 计算城镇化率，并处理可能存在的除以0或空值的情况
df_pop['urbanization_rate'] = df_pop['urban_population'] / df_pop['population']
# 将计算中可能出现的无穷大值替换为空值 (NaN)
df_pop.replace([np.inf, -np.inf], np.nan, inplace=True)

print("人口数据已处理完毕，城镇化率已计算。")


# --- 步骤 3: 从总表的UID中解析出匹配键 ---
uid_str = df_main['uid'].astype(str).str.zfill(4)
df_main['province_code_2digit'] = uid_str.str[:2]
print("已从总表 'uid' 中解析出省份代码。")


# --- 步骤 4: 基于解析出的键，将人口数据合并到总表 ---

# 从人口表中选取需要合并的列
pop_cols_to_merge = ['province_code_2digit', 'year', 'population', 'urban_population', 'urbanization_rate']

# 使用左合并，以总表为基础，确保总表行数不变
final_df = pd.merge(
    df_main,
    df_pop[pop_cols_to_merge],
    on=['province_code_2digit', 'year'],
    how='left'
)
print("\n--- ✨ “人口与城镇化”数据集成完成！ ---")


# --- 步骤 5: 清理和验证 ---
# 删除为合并而创建的临时列
final_df.drop(columns=['province_code_2digit'], inplace=True)

print("最终数据表预览 (请检查 'population', 'urbanization_rate' 等列是否有数据):")
# 随机抽取几行查看合并结果
display(final_df.sample(5))


# --- 步骤 6: 导出最终的CSV文件 ---
output_filename = 'final_data_with_all_controls_v2.csv'
final_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"\n--- ✅ 导出成功！ ---")
print(f"包含所有控制变量的最终面板数据已保存为文件: '{output_filename}'")

--- 开始集成“人口与城镇化”数据 ---
总表和人口数据加载成功。
人口数据已处理完毕，城镇化率已计算。
已从总表 'uid' 中解析出省份代码。

--- ✨ “人口与城镇化”数据集成完成！ ---
最终数据表预览 (请检查 'population', 'urbanization_rate' 等列是否有数据):


,uid,province_id,province,year,co2_emissions,gdp,sec_val,green_invention_patents,green_utility_patents,credit,...,z_equity,z_fund,z_carbon_finance,Factor_1,Factor_2,Factor_3,coal_share_pctg,population,urban_population,urbanization_rate
155,1211,7,Tianjin,2011,123786520,8240.7998,3756.3,791,811,0.034,...,-0.336131,-0.139091,-0.396224,-0.213383,-0.473679,-0.439603,0.307471,1341.00,1090.44,0.813154
42,5318,2,Yunnan,2018,217358240,21427.5000,7267.5,1362,1881,0.043,...,1.276257,0.685114,-0.583868,1.042794,-0.225371,-0.373119,0.496667,4703.00,2309.22,0.491010
218,3702,10,Shandong,2002,450278112,10192.6000,5037.6,130,362,0.024,...,-1.464803,-1.146453,-0.356279,-1.483338,-0.124459,-0.709100,NaN,9082.00,NaN,NaN
50,1502,3,Neimenggu,2002,156339680,1959.7000,754.8,12,28,0.026,...,-1.303564,-1.329610,-0.257706,-1.341156,-0.149465,-0.365970,NaN,2384.05,NaN,NaN
432,4100,19,Henan,2000,223754240,5097.2998,2282.5,55,81,0.022,...,-1.706661,-1.604345,-0.298134,-1.837480,-0.085733,-0.707861,NaN,9488.00,NaN,NaN



--- ✅ 导出成功！ ---
包含所有控制变量的最终面板数据已保存为文件: 'final_data_with_all_controls_v2.csv'


In [9]:
import pandas as pd
import re

# --- 准备工作：定义数据清洗和映射所需的函数与字典 ---

def standardize_province(name):
    """标准化中文省份名称，去除后缀以便合并。"""
    if isinstance(name, str):
        return re.sub(r'省|市|自治区|维吾尔|壮族|回族', '', name).strip()
    return name

# 中文省份名 -> 两位行政区代码 的映射字典
province_codes_map = {
    '北京': '11', '天津': '12', '河北': '13', '山西': '14', '内蒙古': '15', '辽宁': '21',
    '吉林': '22', '黑龙江': '23', '上海': '31', '江苏': '32', '浙江': '33', '安徽': '34',
    '福建': '35', '江西': '36', '山东': '37', '河南': '41', '湖北': '42', '湖南': '43',
    '广东': '44', '广西': '45', '海南': '46', '重庆': '50', '四川': '51', '贵州': '52',
    '云南': '53', '西藏': '54', '陕西': '61', '甘肃': '62', '青海': '63', '宁夏': '64', '新疆': '65'
}

print("--- 开始集成“省级环境保护支出”数据 ---")

# --- 步骤 1: 加载总表和新的环保支出数据 ---
try:
    df_main = pd.read_csv('/Users/yumanlou/Library/CloudStorage/OneDrive-email.cufe.edu.cn/2025/第五学期/论文/ESG/carbon_emission_finance/data/final_data.1.3.1.csv')
    df_env = pd.read_csv('/Users/yumanlou/Library/CloudStorage/OneDrive-email.cufe.edu.cn/2025/第五学期/论文/ESG/carbon_emission_finance/data/eco_ex.csv')
    print("总表和环保支出数据加载成功。")
except FileNotFoundError as e:
    print(f"错误: 找不到文件 {e}。请确保文件都在Notebook所在的文件夹中。")

# --- 步骤 2: 准备“环保支出数据” ---

# 重命名列以便于操作
df_env.rename(columns={
    'id': 'province_chinese',
    '环境保护支出（亿元）': 'env_expenditure',
    '占比': 'env_exp_share'
}, inplace=True)

# 清洗中文省份名
df_env['province_chinese'] = df_env['province_chinese'].apply(standardize_province)

# 根据中文名创建用于匹配的“两位省份代码”
df_env['province_code_2digit'] = df_env['province_chinese'].map(province_codes_map)
print("环保支出数据已处理完毕。")


# --- 步骤 3: 从总表的UID中解析出匹配键 ---
uid_str = df_main['uid'].astype(str).str.zfill(4)
df_main['province_code_2digit'] = uid_str.str[:2]
print("已从总表 'uid' 中解析出省份代码。")


# --- 步骤 4: 基于解析出的键，将环保支出数据合并到总表 ---

# 从环保支出表中选取需要合并的列
env_cols_to_merge = ['province_code_2digit', 'year', 'env_expenditure', 'env_exp_share']

# 使用左合并，以总表为基础，确保总表行数不变
final_df = pd.merge(
    df_main,
    df_env[env_cols_to_merge],
    on=['province_code_2digit', 'year'],
    how='left'
)
print("\n--- ✨ “环保支出”数据集成完成！ ---")


# --- 步骤 5: 清理和验证 ---
# 删除为合并而创建的临时列
final_df.drop(columns=['province_code_2digit'], inplace=True)

print("最终数据表预览 (请检查 'env_expenditure', 'env_exp_share' 列是否有数据):")
# 随机抽取几行查看合并结果
display(final_df.sample(5))


# --- 步骤 6: 导出最终的CSV文件 ---
output_filename = 'final_data_with_all_controls_v3.csv'
final_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"\n--- ✅ 导出成功！ ---")
print(f"包含所有控制变量的最终面板数据已保存为文件: '{output_filename}'")

--- 开始集成“省级环境保护支出”数据 ---
总表和环保支出数据加载成功。
环保支出数据已处理完毕。
已从总表 'uid' 中解析出省份代码。

--- ✨ “环保支出”数据集成完成！ ---
最终数据表预览 (请检查 'env_expenditure', 'env_exp_share' 列是否有数据):


,uid,province_id,province,year,co2_emissions,gdp,sec_val,green_invention_patents,green_utility_patents,credit,...,z_carbon_finance,Factor_1,Factor_2,Factor_3,coal_share_pctg,population,urban_population,urbanization_rate,env_expenditure,env_exp_share
146,1202,7,Tianjin,2002,53101508,1952.3000,931.5,73,83,0.026,...,0.536681,-1.386211,-0.286356,0.009922,NaN,1007.0,NaN,NaN,NaN,NaN
201,3409,9,Anhui,2009,334870304,11063.3000,4918.7,339,385,0.036,...,-0.364494,-0.160355,-0.482496,-0.430335,0.616359,6131.0,2581.15,0.421000,59.2653,0.027669
589,3513,25,Fujian,2013,232099104,22851.6990,11805.5,791,1240,0.040,...,-0.422317,0.623218,-0.595876,-0.366099,0.345865,3885.0,2293.46,0.590337,58.6029,0.019096
181,6413,8,Ningxia,2013,128478800,2364.8999,1059.5,166,92,0.039,...,-0.139643,0.676593,-0.811890,0.322510,0.477805,666.0,340.24,0.510871,32.9285,0.035696
32,5308,2,Yunnan,2008,165360704,6120.6001,2476.1,163,114,0.035,...,-0.493348,-0.274499,-0.463394,-0.453599,0.626677,4543.0,1499.19,0.330000,58.4582,0.039761



--- ✅ 导出成功！ ---
包含所有控制变量的最终面板数据已保存为文件: 'final_data_with_all_controls_v3.csv'


In [10]:
import pandas as pd
import re

# --- 准备工作：定义数据清洗和映射所需的函数与字典 ---

def standardize_province(name):
    """标准化中文省份名称，去除后缀以便合并。"""
    if isinstance(name, str):
        return re.sub(r'省|市|自治区|维吾尔|壮族|回族', '', name).strip()
    return name

# 中文省份名 -> 两位行政区代码 的映射字典
province_codes_map = {
    '北京': '11', '天津': '12', '河北': '13', '山西': '14', '内蒙古': '15', '辽宁': '21',
    '吉林': '22', '黑龙江': '23', '上海': '31', '江苏': '32', '浙江': '33', '安徽': '34',
    '福建': '35', '江西': '36', '山东': '37', '河南': '41', '湖北': '42', '湖南': '43',
    '广东': '44', '广西': '45', '海南': '46', '重庆': '50', '四川': '51', '贵州': '52',
    '云南': '53', '西藏': '54', '陕西': '61', '甘肃': '62', '青海': '63', '宁夏': '64', '新疆': '65'
}

print("--- 开始集成“工业污染物排放”数据 ---")

# --- 步骤 1: 加载总表和包含污染物数据的文件 ---
try:
    df_main = pd.read_csv('/Users/yumanlou/Library/CloudStorage/OneDrive-email.cufe.edu.cn/2025/第五学期/论文/ESG/carbon_emission_finance/data/final_data.1.3.2_did.csv')
    # 污染物数据与GTFP在同一个文件里
    df_pollutants = pd.read_csv('/Users/yumanlou/Library/CloudStorage/OneDrive-email.cufe.edu.cn/2025/第五学期/论文/ESG/carbon_emission_finance/data/GTMP.csv')
    print("总表和污染物数据加载成功。")
except FileNotFoundError as e:
    print(f"错误: 找不到文件 {e}。请确保文件都在Notebook所在的文件夹中。")

# --- 步骤 2: 准备“污染物数据” ---

# 定义需要抽取的列和新的列名
cols_to_use = {
    '地区': 'province_chinese',
    '年份': 'year',
    '工业废水排放量（万吨）': 'industrial_wastewater',
    '工业二氧化硫排放量（万吨）': 'industrial_so2',
    '工业固体废弃物排放量（万吨）': 'industrial_solid_waste'
}

# 选取并重命名列
df_pollutants = df_pollutants[list(cols_to_use.keys())].rename(columns=cols_to_use)

# 清洗中文省份名
df_pollutants['province_chinese'] = df_pollutants['province_chinese'].apply(standardize_province)

# 根据中文名创建用于匹配的“两位省份代码”
df_pollutants['province_code_2digit'] = df_pollutants['province_chinese'].map(province_codes_map)
print("污染物数据已处理完毕。")


# --- 步骤 3: 从总表的UID中解析出匹配键 ---
uid_str = df_main['uid'].astype(str).str.zfill(4)
df_main['province_code_2digit'] = uid_str.str[:2]
print("已从总表 'uid' 中解析出省份代码。")


# --- 步骤 4: 基于解析出的键，将污染物数据合并到总表 ---

# 从污染物表中选取需要合并的列
pollutants_cols_to_merge = ['province_code_2digit', 'year', 'industrial_wastewater', 'industrial_so2', 'industrial_solid_waste']

# 使用左合并，以总表为基础
final_df = pd.merge(
    df_main,
    df_pollutants[pollutants_cols_to_merge],
    on=['province_code_2digit', 'year'],
    how='left'
)
print("\n--- ✨ 工业污染物数据集成完成！ ---")


# --- 步骤 5: 清理和验证 ---
# 删除为合并而创建的临时列
final_df.drop(columns=['province_code_2digit'], inplace=True)

print("最终数据表预览 (请检查 'industrial_so2' 等新列是否有数据):")
# 筛选山西省的数据，查看合并是否成功
display(final_df[final_df['province'] == 'Shanxi'].tail())


# --- 步骤 6: 导出最终的CSV文件 ---
output_filename = 'final_data_with_all_variables.csv'
final_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"\n--- ✅ 导出成功！ ---")
print(f"包含所有变量的最终面板数据已保存为文件: '{output_filename}'")

--- 开始集成“工业污染物排放”数据 ---
总表和污染物数据加载成功。
污染物数据已处理完毕。
已从总表 'uid' 中解析出省份代码。

--- ✨ 工业污染物数据集成完成！ ---
最终数据表预览 (请检查 'industrial_so2' 等新列是否有数据):


/var/folders/8m/nz317wg157v0xygc8d7t4_pw0000gn/T/ipykernel_21408/436529234.py:27: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pollutants = pd.read_csv('/Users/yumanlou/Library/CloudStorage/OneDrive-email.cufe.edu.cn/2025/第五学期/论文/ESG/carbon_emission_finance/data/GTMP.csv')


,uid,province_id,province,year,co2_emissions,gdp,sec_val,green_invention_patents,green_utility_patents,credit,...,post_1,post_2,did_pre2,did_post0,did_post1,did_post2,sec_pctg,industrial_wastewater,industrial_so2,industrial_solid_waste
259,1419,11,Shanxi,2019,529401536,17311.000,7402.8999,947,1825,0.046,...,0,0,1,0,0,0,0.427641,14394.000,19.716,30.339
260,1420,11,Shanxi,2020,530238208,18202.699,7440.2002,1117,2397,0.047,...,0,0,0,0,0,0,0.408742,15859.000,12.249,35.299
261,1421,11,Shanxi,2021,579007296,23087.801,10892.5000,1130,2023,0.052,...,0,0,0,1,0,0,0.471786,18603.000,10.381,18.533
262,1422,11,Shanxi,2022,589578368,25653.199,12667.1000,1232,1789,0.051,...,1,0,0,0,1,0,0.493782,17236.802,9.087,17.289
263,1423,11,Shanxi,2023,629094784,26050.801,11940.1000,1543,1941,0.052,...,0,1,0,0,0,1,0.458339,16179.614,7.262,16.117



--- ✅ 导出成功！ ---
包含所有变量的最终面板数据已保存为文件: 'final_data_with_all_variables.csv'


In [11]:
import pandas as pd
import re

# --- 准备工作：定义数据清洗和映射所需的函数与字典 ---

def standardize_province(name):
    """标准化中文省份名称，去除后缀以便合并。"""
    if isinstance(name, str):
        return re.sub(r'省|市|自治区|维吾尔|壮族|回族', '', name).strip()
    return name

# 中文省份名 -> 两位行政区代码 的映射字典
province_codes_map = {
    '北京': '11', '天津': '12', '河北': '13', '山西': '14', '内蒙古': '15', '辽宁': '21',
    '吉林': '22', '黑龙江': '23', '上海': '31', '江苏': '32', '浙江': '33', '安徽': '34',
    '福建': '35', '江西': '36', '山东': '37', '河南': '41', '湖北': '42', '湖南': '43',
    '广东': '44', '广西': '45', '海南': '46', '重庆': '50', '四川': '51', '贵州': '52',
    '云南': '53', '西藏': '54', '陕西': '61', '甘肃': '62', '青海': '63', '宁夏': '64', '新疆': '65'
}

print("--- 开始集成“工业绿色全要素生产率 (GTFP)”数据 ---")

# --- 步骤 1: 加载总表和包含GTFP数据的文件 ---
try:
    df_main = pd.read_csv('/Users/yumanlou/Library/CloudStorage/OneDrive-email.cufe.edu.cn/2025/第五学期/论文/ESG/carbon_emission_finance/data/final_data.1.3.3_did.csv')
    # 使用 low_memory=False 来避免您之前遇到的 DtypeWarning 警告
    df_gtfp = pd.read_csv('/Users/yumanlou/Library/CloudStorage/OneDrive-email.cufe.edu.cn/2025/第五学期/论文/ESG/carbon_emission_finance/data/GTMP.csv', low_memory=False)
    print("总表和GTFP数据加载成功。")
except FileNotFoundError as e:
    print(f"错误: 找不到文件 {e}。请确保文件都在Notebook所在的文件夹中。")

# --- 步骤 2: 准备“GTFP数据” ---

# 定义需要抽取的列和新的列名
cols_to_use = {
    '地区': 'province_chinese',
    '年份': 'year',
    '工业绿色全要素生产率': 'gtfp_level',
    'GML指数': 'gml_index' # GML指数是GTFP的变动率，也一并加入
}

# 选取并重命名列
df_gtfp = df_gtfp[list(cols_to_use.keys())].rename(columns=cols_to_use)

# 清洗中文省份名
df_gtfp['province_chinese'] = df_gtfp['province_chinese'].apply(standardize_province)

# 根据中文名创建用于匹配的“两位省份代码”
df_gtfp['province_code_2digit'] = df_gtfp['province_chinese'].map(province_codes_map)
print("GTFP数据已处理完毕。")


# --- 步骤 3: 从总表的UID中解析出匹配键 ---
uid_str = df_main['uid'].astype(str).str.zfill(4)
df_main['province_code_2digit'] = uid_str.str[:2]
print("已从总表 'uid' 中解析出省份代码。")


# --- 步骤 4: 基于解析出的键，将GTFP数据合并到总表 ---

# 从GTFP表中选取需要合并的列
gtfp_cols_to_merge = ['province_code_2digit', 'year', 'gtfp_level', 'gml_index']

# 使用左合并，以总表为基础
final_df = pd.merge(
    df_main,
    df_gtfp[gtfp_cols_to_merge],
    on=['province_code_2digit', 'year'],
    how='left'
)
print("\n--- ✨ GTFP数据集成完成！ ---")


# --- 步骤 5: 清理和验证 ---
# 删除为合并而创建的临时列
final_df.drop(columns=['province_code_2digit'], inplace=True)

print("最终数据表预览 (请检查 'gtfp_level' 和 'gml_index' 列是否有数据):")
# 筛选山西省的数据，查看合并是否成功
display(final_df[final_df['province'] == 'Shanxi'].tail())


# --- 步骤 6: 导出最终的CSV文件 ---
output_filename = 'final_data_with_all_variables_v2.csv'
final_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"\n--- ✅ 导出成功！ ---")
print(f"包含所有变量（包括GTFP）的最终面板数据已保存为文件: '{output_filename}'")

--- 开始集成“工业绿色全要素生产率 (GTFP)”数据 ---
总表和GTFP数据加载成功。
GTFP数据已处理完毕。
已从总表 'uid' 中解析出省份代码。

--- ✨ GTFP数据集成完成！ ---
最终数据表预览 (请检查 'gtfp_level' 和 'gml_index' 列是否有数据):


,uid,province_id,province,year,co2_emissions,gdp,sec_val,green_invention_patents,green_utility_patents,credit,...,did_pre2,did_post0,did_post1,did_post2,sec_pctg,industrial_wastewater,industrial_so2,industrial_solid_waste,gtfp_level,gml_index
259,1419,11,Shanxi,2019,529401536,17311.000,7402.8999,947,1825,0.046,...,1,0,0,0,0.427641,14394.000,19.716,30.339,2.962793,1.158634
260,1420,11,Shanxi,2020,530238208,18202.699,7440.2002,1117,2397,0.047,...,0,0,0,0,0.408742,15859.000,12.249,35.299,2.837600,0.957745
261,1421,11,Shanxi,2021,579007296,23087.801,10892.5000,1130,2023,0.052,...,0,1,0,0,0.471786,18603.000,10.381,18.533,4.350216,1.533062
262,1422,11,Shanxi,2022,589578368,25653.199,12667.1000,1232,1789,0.051,...,0,0,1,0,0.493782,17236.802,9.087,17.289,8.940098,2.055093
263,1423,11,Shanxi,2023,629094784,26050.801,11940.1000,1543,1941,0.052,...,0,0,0,1,0.458339,16179.614,7.262,16.117,8.917353,0.997456



--- ✅ 导出成功！ ---
包含所有变量（包括GTFP）的最终面板数据已保存为文件: 'final_data_with_all_variables_v2.csv'


In [12]:
import pandas as pd
import re

# --- 准备工作：定义数据清洗和映射所需的函数与字典 ---

def standardize_province(name):
    """标准化中文省份名称，去除后缀以便合并。"""
    if isinstance(name, str):
        return re.sub(r'省|市|自治区|维吾尔|壮族|回族', '', name).strip()
    return name

# 中文省份名 -> 两位行政区代码 的映射字典
province_codes_map = {
    '北京': '11', '天津': '12', '河北': '13', '山西': '14', '内蒙古': '15', '辽宁': '21',
    '吉林': '22', '黑龙江': '23', '上海': '31', '江苏': '32', '浙江': '33', '安徽': '34',
    '福建': '35', '江西': '36', '山东': '37', '河南': '41', '湖北': '42', '湖南': '43',
    '广东': '44', '广西': '45', '海南': '46', '重庆': '50', '四川': '51', '贵州': '52',
    '云南': '53', '西藏': '54', '陕西': '61', '甘肃': '62', '青海': '63', '宁夏': '64', '新疆': '65'
}

print("--- 开始集成“市场化水平”数据 ---")

# --- 步骤 1: 加载总表和新的市场化指数数据 ---
try:
    df_main = pd.read_csv('/Users/yumanlou/Library/CloudStorage/OneDrive-email.cufe.edu.cn/2025/第五学期/论文/ESG/carbon_emission_finance/data/final_data.1.3.3_did.csv')
    df_market = pd.read_csv('/Users/yumanlou/Library/CloudStorage/OneDrive-email.cufe.edu.cn/2025/第五学期/论文/ESG/carbon_emission_finance/data/mkt.csv')
    print("总表和市场化指数数据加载成功。")
except FileNotFoundError as e:
    print(f"错误: 找不到文件 {e}。请确保文件都在Notebook所在的文件夹中。")

# --- 步骤 2: 准备“市场化数据” ---

# 重命名列以便于操作和理解
df_market.rename(columns={
    '省份': 'province_chinese',
    'market': 'market_index', # 市场化总指数
    '政府与市场关系': 'market_gov_relation',
    '非国有经济发展': 'market_npoe_dev',
    '产品市场的发育程度': 'market_product_dev',
    '要素市场的发育程度': 'market_factor_dev',
    '市场中介组织的发育和法律制度环境': 'market_legal_env'
}, inplace=True)


# 清洗中文省份名
df_market['province_chinese'] = df_market['province_chinese'].apply(standardize_province)

# 根据中文名创建用于匹配的“两位省份代码”
df_market['province_code_2digit'] = df_market['province_chinese'].map(province_codes_map)
print("市场化指数数据已处理完毕。")


# --- 步骤 3: 从总表的UID中解析出匹配键 ---
uid_str = df_main['uid'].astype(str).str.zfill(4)
df_main['province_code_2digit'] = uid_str.str[:2]
print("已从总表 'uid' 中解析出省份代码。")


# --- 步骤 4: 基于解析出的键，将市场化指数数据合并到总表 ---

# 从市场化指数表中选取需要合并的列
market_cols_to_merge = [
    'province_code_2digit', 'year', 'market_index', 'market_gov_relation',
    'market_npoe_dev', 'market_product_dev', 'market_factor_dev', 'market_legal_env'
]

# 使用左合并，以总表为基础
final_df = pd.merge(
    df_main,
    df_market[market_cols_to_merge],
    on=['province_code_2digit', 'year'],
    how='left'
)
print("\n--- ✨ 市场化指数数据集成完成！ ---")


# --- 步骤 5: 清理和验证 ---
# 删除为合并而创建的临时列
final_df.drop(columns=['province_code_2digit'], inplace=True)

print("最终数据表预览 (请检查 'market_index' 等新列是否有数据):")
# 筛选山西省的数据，查看合并是否成功
display(final_df[final_df['province'] == 'Shanxi'].tail())


# --- 步骤 6: 导出最终的CSV文件 ---
output_filename = 'final_master_dataset.csv'
final_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"\n--- ✅ 导出成功！ ---")
print(f"您的最终版、包含所有变量的主数据表已保存为文件: '{output_filename}'")

--- 开始集成“市场化水平”数据 ---
总表和市场化指数数据加载成功。
市场化指数数据已处理完毕。
已从总表 'uid' 中解析出省份代码。

--- ✨ 市场化指数数据集成完成！ ---
最终数据表预览 (请检查 'market_index' 等新列是否有数据):


,uid,province_id,province,year,co2_emissions,gdp,sec_val,green_invention_patents,green_utility_patents,credit,...,industrial_so2,industrial_solid_waste,gtfp_level,gml_index,market_index,market_gov_relation,market_npoe_dev,market_product_dev,market_factor_dev,market_legal_env
259,1419,11,Shanxi,2019,529401536,17311.000,7402.8999,947,1825,0.046,...,19.716,30.339,2.962793,1.158634,6.74,5.56,9.394,2.124,8.18,8.44
260,1420,11,Shanxi,2020,530238208,18202.699,7440.2002,1117,2397,0.047,...,12.249,35.299,2.837600,0.957745,6.74,5.56,9.394,2.124,8.18,8.44
261,1421,11,Shanxi,2021,579007296,23087.801,10892.5000,1130,2023,0.052,...,10.381,18.533,4.350216,1.533062,6.74,5.56,9.394,2.124,8.18,8.44
262,1422,11,Shanxi,2022,589578368,25653.199,12667.1000,1232,1789,0.051,...,9.087,17.289,8.940098,2.055093,6.74,5.56,9.394,2.124,8.18,8.44
263,1423,11,Shanxi,2023,629094784,26050.801,11940.1000,1543,1941,0.052,...,7.262,16.117,8.917353,0.997456,6.74,5.56,9.394,2.124,8.18,8.44



--- ✅ 导出成功！ ---
您的最终版、包含所有变量的主数据表已保存为文件: 'final_master_dataset.csv'
